# Xor with Pytorch: Dataset and DataLoader

We now add a `Dataset` and wrap it in a `Dataloader`

**Note:** Obviously, creating a custom Dataset is not necesary for such a simple network, this is only for demo/learning pourposes

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# Imports
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from pathlib import Path
from torch.utils.data import Dataset, DataLoader

Create dataset

In [3]:
class DatasetXor(Dataset):
    """ 
    Custom dataset: We have to override methods __len__ and __getitem__
    """
    def __init__(self, num):
        x1 = np.random.randn(num)
        x2 = np.random.randn(num)
        x = np.stack([x1, x2], axis=1).astype('float32')
        y = (2.0 * np.logical_xor(x1 >= 0, x2 >= 0).astype('float32') - 1.0) * 0.9
        self.x, self.y = torch.Tensor(x), torch.Tensor(y)
    
    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return (self.x[idx], self.y[idx])

Create model

In [4]:
# Module class example. It does exactly the same as:
#     nn.Sequential(nn.Linear(...), nn.Tanh(), nn.Linear(...), nn.Tanh())
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer_1 = nn.Linear(2,3)
        self.layer_2 = nn.Linear(3,1)

    def forward(self, x):
        x = self.layer_1(x)
        x = torch.tanh(x)
        x = self.layer_2(x)
        out = torch.tanh(x)
        return out

Train model: Now we can use Dataloader's batch

In [5]:
def train(model, dataloader, epochs, lr, momentum):
    device = torch.device("cpu")
    model.train()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
    for n_epoch in range(epochs):
        for n_batch, batch in enumerate(dataloader):
            x, y = batch
            optimizer.zero_grad()
            output = model(x)
            loss = F.mse_loss(output.squeeze(), y)
            loss.backward()
            optimizer.step()
            if n_epoch % 50 == 0 and n_batch == 0:
                print(f"Train Epoch: {n_epoch} / {epochs}\tn_batch: {n_batch}\tLoss: {loss.item():.6f}")

In [8]:
dataset_xor = DatasetXor(1000)
dataloader = DataLoader(dataset_xor, batch_size=100)
model = Net()
train(model, dataloader, epochs=1000, lr=0.05, momentum=0.8)

Train Epoch: 0 / 1000	n_batch: 0	Loss: 1.202278
Train Epoch: 50 / 1000	n_batch: 0	Loss: 0.251538
Train Epoch: 100 / 1000	n_batch: 0	Loss: 0.218983
Train Epoch: 150 / 1000	n_batch: 0	Loss: 0.201405
Train Epoch: 200 / 1000	n_batch: 0	Loss: 0.193140
Train Epoch: 250 / 1000	n_batch: 0	Loss: 0.187015
Train Epoch: 300 / 1000	n_batch: 0	Loss: 0.181673
Train Epoch: 350 / 1000	n_batch: 0	Loss: 0.177206
Train Epoch: 400 / 1000	n_batch: 0	Loss: 0.173693
Train Epoch: 450 / 1000	n_batch: 0	Loss: 0.170972
Train Epoch: 500 / 1000	n_batch: 0	Loss: 0.168836
Train Epoch: 550 / 1000	n_batch: 0	Loss: 0.167122
Train Epoch: 600 / 1000	n_batch: 0	Loss: 0.165717
Train Epoch: 650 / 1000	n_batch: 0	Loss: 0.164542
Train Epoch: 700 / 1000	n_batch: 0	Loss: 0.163545
Train Epoch: 750 / 1000	n_batch: 0	Loss: 0.162686
Train Epoch: 800 / 1000	n_batch: 0	Loss: 0.161940
Train Epoch: 850 / 1000	n_batch: 0	Loss: 0.161283
Train Epoch: 900 / 1000	n_batch: 0	Loss: 0.160700
Train Epoch: 950 / 1000	n_batch: 0	Loss: 0.160177


In [9]:
x_train, y_train = dataset_xor.x, dataset_xor.y
yhat = model(x_train)
for i in range(len(yhat)):
    print(f"{y_train[i]}\t{yhat[i]}\t{x_train[i]}")

0.8999999761581421	tensor([0.7346], grad_fn=<SelectBackward>)	tensor([ 0.1616, -0.7934])
0.8999999761581421	tensor([0.5782], grad_fn=<SelectBackward>)	tensor([-0.3324,  0.5263])
0.8999999761581421	tensor([0.9882], grad_fn=<SelectBackward>)	tensor([ 1.3072, -0.5032])
-0.8999999761581421	tensor([-0.4220], grad_fn=<SelectBackward>)	tensor([0.5748, 0.0067])
0.8999999761581421	tensor([-0.7075], grad_fn=<SelectBackward>)	tensor([ 0.0582, -2.0945])
0.8999999761581421	tensor([0.9770], grad_fn=<SelectBackward>)	tensor([ 0.5072, -0.7115])
-0.8999999761581421	tensor([-0.9556], grad_fn=<SelectBackward>)	tensor([-1.8907, -1.1618])
-0.8999999761581421	tensor([-0.9379], grad_fn=<SelectBackward>)	tensor([1.2577, 1.2039])
0.8999999761581421	tensor([1.], grad_fn=<SelectBackward>)	tensor([-2.0912,  3.2852])
-0.8999999761581421	tensor([-1.0000], grad_fn=<SelectBackward>)	tensor([-1.2281, -0.7981])
-0.8999999761581421	tensor([-0.6810], grad_fn=<SelectBackward>)	tensor([0.2395, 0.6179])
0.8999999761581421	t

0.8999999761581421	tensor([1.0000], grad_fn=<SelectBackward>)	tensor([-0.8080,  1.5859])
-0.8999999761581421	tensor([-0.8178], grad_fn=<SelectBackward>)	tensor([0.6558, 0.4763])
-0.8999999761581421	tensor([-0.7502], grad_fn=<SelectBackward>)	tensor([-1.1371, -2.0582])
-0.8999999761581421	tensor([-0.7852], grad_fn=<SelectBackward>)	tensor([-0.3823, -0.2963])
-0.8999999761581421	tensor([-0.6868], grad_fn=<SelectBackward>)	tensor([-0.2277, -0.3751])
-0.8999999761581421	tensor([-0.9135], grad_fn=<SelectBackward>)	tensor([1.5532, 0.7961])
-0.8999999761581421	tensor([-0.8414], grad_fn=<SelectBackward>)	tensor([0.7897, 0.5051])
-0.8999999761581421	tensor([-0.8730], grad_fn=<SelectBackward>)	tensor([0.7231, 0.7174])
0.8999999761581421	tensor([1.0000], grad_fn=<SelectBackward>)	tensor([-1.4670,  0.5287])
0.8999999761581421	tensor([0.2619], grad_fn=<SelectBackward>)	tensor([ 0.0593, -1.0680])
0.8999999761581421	tensor([-0.3042], grad_fn=<SelectBackward>)	tensor([-0.0679,  0.1412])
-0.89999997615

0.8999999761581421	tensor([0.1338], grad_fn=<SelectBackward>)	tensor([ 0.6490, -0.1938])
0.8999999761581421	tensor([0.9986], grad_fn=<SelectBackward>)	tensor([-0.5588,  1.5230])
-0.8999999761581421	tensor([-0.9980], grad_fn=<SelectBackward>)	tensor([-0.4700, -0.6643])
0.8999999761581421	tensor([0.9109], grad_fn=<SelectBackward>)	tensor([-0.4749,  0.6608])
0.8999999761581421	tensor([0.9999], grad_fn=<SelectBackward>)	tensor([ 0.9890, -2.4494])
-0.8999999761581421	tensor([-0.7780], grad_fn=<SelectBackward>)	tensor([0.5192, 2.0449])
0.8999999761581421	tensor([0.9495], grad_fn=<SelectBackward>)	tensor([-0.3983,  1.1500])
-0.8999999761581421	tensor([-0.7645], grad_fn=<SelectBackward>)	tensor([0.4085, 1.5236])
0.8999999761581421	tensor([0.9998], grad_fn=<SelectBackward>)	tensor([-0.5273,  2.2567])
-0.8999999761581421	tensor([-1.0000], grad_fn=<SelectBackward>)	tensor([-0.6060, -0.8246])
0.8999999761581421	tensor([0.8207], grad_fn=<SelectBackward>)	tensor([ 0.9236, -0.3688])
-0.89999997615814

-0.8999999761581421	tensor([-0.9999], grad_fn=<SelectBackward>)	tensor([-0.4598, -1.0595])
-0.8999999761581421	tensor([-1.0000], grad_fn=<SelectBackward>)	tensor([-0.5426, -0.9528])
0.8999999761581421	tensor([0.9721], grad_fn=<SelectBackward>)	tensor([-0.6729,  0.4607])
0.8999999761581421	tensor([0.9972], grad_fn=<SelectBackward>)	tensor([-1.2197,  0.2916])
0.8999999761581421	tensor([1.0000], grad_fn=<SelectBackward>)	tensor([ 1.0773, -1.3310])
0.8999999761581421	tensor([-0.2064], grad_fn=<SelectBackward>)	tensor([-0.0075,  0.9132])
-0.8999999761581421	tensor([-0.4725], grad_fn=<SelectBackward>)	tensor([0.8929, 0.0648])
0.8999999761581421	tensor([1.0000], grad_fn=<SelectBackward>)	tensor([ 1.2698, -0.9139])
-0.8999999761581421	tensor([-0.9494], grad_fn=<SelectBackward>)	tensor([1.3986, 1.7871])
0.8999999761581421	tensor([-0.4047], grad_fn=<SelectBackward>)	tensor([ 0.3366, -0.0130])
-0.8999999761581421	tensor([-1.0000], grad_fn=<SelectBackward>)	tensor([-1.0219, -1.3580])
0.89999997615

0.8999999761581421	tensor([0.9884], grad_fn=<SelectBackward>)	tensor([-0.6363,  0.6954])
-0.8999999761581421	tensor([0.4383], grad_fn=<SelectBackward>)	tensor([0.1678, 2.6935])
0.8999999761581421	tensor([0.8351], grad_fn=<SelectBackward>)	tensor([ 0.2671, -0.6855])
0.8999999761581421	tensor([1.0000], grad_fn=<SelectBackward>)	tensor([-1.2670,  0.9113])
0.8999999761581421	tensor([0.3968], grad_fn=<SelectBackward>)	tensor([ 1.1206, -0.1730])
-0.8999999761581421	tensor([-0.3322], grad_fn=<SelectBackward>)	tensor([-0.0533, -0.1775])
-0.8999999761581421	tensor([-0.9883], grad_fn=<SelectBackward>)	tensor([-0.1763, -1.1755])
-0.8999999761581421	tensor([-0.8379], grad_fn=<SelectBackward>)	tensor([-0.3212, -0.3929])
0.8999999761581421	tensor([-0.0953], grad_fn=<SelectBackward>)	tensor([ 0.8242, -0.0913])
-0.8999999761581421	tensor([-0.6540], grad_fn=<SelectBackward>)	tensor([-0.0432, -1.0313])
0.8999999761581421	tensor([1.0000], grad_fn=<SelectBackward>)	tensor([ 0.5593, -1.2597])
-0.8999999761